In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test)

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [2]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

## LightGBM model

In [3]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.8988911725316586,
 'bagging_freq': 22.0,
 'feature_fraction': 0.6622442122619671,
 'learning_rate': 0.16422725363286422,
 'max_depth': 22.0,
 'num_leaves': 180.0,
 'test_size': 0.20892455926004772}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [4]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

Using TensorFlow backend.


## XGBoost

In [5]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [6]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer


features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'varianza_id_zona', 'tam_ambientes', 'metros_cubiertos_normalizados', 'dias_desde_datos',
                 'meses_desde_datos', 'promedio_id_zona_log']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('xgboost', xgb_m), 
#            ('keras', keras_m), 
           ('lightgbm', lgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2, n_folds=6)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBoostWrapper]
[23:27:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:2.19093e+06
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:1.89164e+06
[2]	validation_0-mae:1.63443e+06
[3]	validation_0-mae:1.41305e+06
[4]	validation_0-mae:1.22332e+06
[5]	validation_0-mae:1.0611e+06
[6]	validation_0-mae:922343
[7]	validation_0-mae:803753
[8]	validation_0-mae:702876
[9]	validation_0-mae:617544
[10]	validation_0-mae:545488
[11]	validation_0-mae:485060
[12]	validation_0-mae:434088
[13]	validation_0-mae:391476
[14]	validation_0-mae:355909
[15]	validation_0-mae:326377
[16]	validation_0-mae:301630
[17]	validation_0-mae:280903
[18]	validation_0-mae:262540
[19]	validation_0-mae:247615
[20]	validation_0-mae:234889
[21]	validation_0-mae:224246
[22]	validati

[107]	validation_0-mae:69672.7
[108]	validation_0-mae:69460.6
[109]	validation_0-mae:69192.7
[110]	validation_0-mae:68867.2
[111]	validation_0-mae:68413.9
[112]	validation_0-mae:68192.5
[113]	validation_0-mae:67930.9
[114]	validation_0-mae:67735.9
[115]	validation_0-mae:67641.2
[116]	validation_0-mae:67340.4
[117]	validation_0-mae:66843.9
[118]	validation_0-mae:66535.2
[119]	validation_0-mae:66129.2
[120]	validation_0-mae:65648.8
[121]	validation_0-mae:65459.6
[122]	validation_0-mae:65233
[123]	validation_0-mae:65003.7
[124]	validation_0-mae:64905.8
[125]	validation_0-mae:64479.8
[126]	validation_0-mae:64259
[127]	validation_0-mae:63910.9
[128]	validation_0-mae:63491.5
[129]	validation_0-mae:63391.7
[130]	validation_0-mae:63122.8
[131]	validation_0-mae:62963.6
[132]	validation_0-mae:62808.8
[133]	validation_0-mae:62707.9
[134]	validation_0-mae:62485.9
[135]	validation_0-mae:62405.2
[136]	validation_0-mae:61916.9
[137]	validation_0-mae:61629
[138]	validation_0-mae:61161.6
[139]	validati

[67]	validation_0-mae:96678.6
[68]	validation_0-mae:96414.2
[69]	validation_0-mae:95815.9
[70]	validation_0-mae:94931.9
[71]	validation_0-mae:94598.7
[72]	validation_0-mae:93843.5
[73]	validation_0-mae:93173.7
[74]	validation_0-mae:92642.1
[75]	validation_0-mae:92234.7
[76]	validation_0-mae:91080
[77]	validation_0-mae:90831.2
[78]	validation_0-mae:90266.8
[79]	validation_0-mae:89742.1
[80]	validation_0-mae:89186.8
[81]	validation_0-mae:88839.4
[82]	validation_0-mae:87956.5
[83]	validation_0-mae:87497.9
[84]	validation_0-mae:87310.7
[85]	validation_0-mae:87014.8
[86]	validation_0-mae:86461.5
[87]	validation_0-mae:86238.6
[88]	validation_0-mae:85955.1
[89]	validation_0-mae:85664
[90]	validation_0-mae:85039.9
[91]	validation_0-mae:84810.9
[92]	validation_0-mae:84398.1
[93]	validation_0-mae:83961.1
[94]	validation_0-mae:83395.1
[95]	validation_0-mae:82884.9
[96]	validation_0-mae:82144.4
[97]	validation_0-mae:80963.3
[98]	validation_0-mae:79927.3
[99]	validation_0-mae:79092.2
[100]	validati

[25]	validation_0-mae:190998
[26]	validation_0-mae:184913
[27]	validation_0-mae:178699
[28]	validation_0-mae:173216
[29]	validation_0-mae:168624
[30]	validation_0-mae:163949
[31]	validation_0-mae:159520
[32]	validation_0-mae:156560
[33]	validation_0-mae:153028
[34]	validation_0-mae:149929
[35]	validation_0-mae:146395
[36]	validation_0-mae:142764
[37]	validation_0-mae:139315
[38]	validation_0-mae:136272
[39]	validation_0-mae:134054
[40]	validation_0-mae:131876
[41]	validation_0-mae:129324
[42]	validation_0-mae:127006
[43]	validation_0-mae:123955
[44]	validation_0-mae:121778
[45]	validation_0-mae:119385
[46]	validation_0-mae:117864
[47]	validation_0-mae:115753
[48]	validation_0-mae:114156
[49]	validation_0-mae:111807
[50]	validation_0-mae:109703
[51]	validation_0-mae:108446
[52]	validation_0-mae:106437
[53]	validation_0-mae:105728
[54]	validation_0-mae:104895
[55]	validation_0-mae:103054
[56]	validation_0-mae:102015
[57]	validation_0-mae:100765
[58]	validation_0-mae:99561.5
[59]	validati

In [8]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_test_f, cols_tipodepropiedad_ohe = columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)

df_train_f = columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [7]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [8]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [9]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [10]:
params_2nd = {'bagging_fraction': 0.7740520226030885,
 'bagging_freq': int(7.0),
 'feature_fraction': 0.8422472893793045,
 'learning_rate': 0.1508386725397851,
 'max_depth': int(12.0),
 'num_leaves': int(110.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

LightGBMWrapper(bagging_fraction=0.7740520226030885, bagging_freq=7,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.8422472893793045, importance_type='split',
                learning_rate=0.1508386725397851, max_depth=12,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=110,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [11]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02']))
df_test_s[['id', 'target']].to_csv('respuesta20.csv', index = False)

## Prediccion solo con features de stacking

In [17]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features)), df_train_f['precio'].values)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



LightGBMWrapper(bagging_fraction=0.8924398062087346, bagging_freq=36,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.16167385124183287, importance_type='split',
                learning_rate=0.054693418899570134, max_depth=4,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=93,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [24]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

MAE St

In [26]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta21.csv', index = False)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



In [13]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

 12%|█▏        | 12/100 [00:48<05:53,  4.02s/it, best loss: 437714.00137358136]


KeyboardInterrupt: 

In [141]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")

MAE Keras (train): 524925.45271
